## UPDATES IN PROGRESS -- PRIOR SAVED VERSIONS OF FINAL DATASETS CAN BE LOADED IN TRAINING MODELS WORKSHEET

In [1]:
### Things to do:
### - Add in current game situation (inning, score diff, runners on)
### Right now rolling on a season and a month, but can exp with other time periods like 2 or 3 months, or 3, 7 games
### - Add in num batters faced for pitcher in game or inning (also maybe a rest metric like days since last pitched if its easy)
### Figure out why unnamed cols get added every pull and push from google cloud and stop that from happening (takes up data space)
### I wonder if we might undercount runs due to not all errors being counted as such. We could calculate the odds of runners advancing on things like out and fly out (non3b) to stop this.
### I wonder if we're potentially overcounting whatever the most recent play(s) for individual players are, as we roll by at bats rather than rolling at bats every game or even week. For example, if you do play x, particularly in a smaller sample, the percentage immedietly jumps and slowly lowers until it happens again. Additionally, when simulating we (should) update percentages with each at bat in game if we do this in training...



# Use the below to eventually build a stealing df to help with the simulation
# pitches[pitches.des.str.contains("steal") == True].iloc[40]

### Import Packages and Warnings

In [1]:
# Major Packages
import sys

# Modules in folder
import constants
from functions import *

# Warnings
warnings.simplefilter("ignore")

# Internal Modules Outside Current Folder
# Change the path so that we can import the local cloud functions stored in a different directory. THE PATH IS DIFFERENT ON MAC AND PI, SO USE TRY EXCEPT FOR BOTH
try: # for mac
    sys.path.insert(1, '/users/jaredzirkes/Desktop/Python/GitHub')
    from google_cloud.cloud_functions import CloudHelper
except:
    sys.path.insert(1, "/home/pi/Desktop/Python")
    from google_cloud.cloud_functions import CloudHelper

YEAR = 2016

### Import Prior Data

In [2]:
# Import file of raw pitches from the given year
all_pitches = CloudHelper().download_from_cloud("yearly_pitches_files/pitches_{}.pkl".format(YEAR))


In [3]:
z = all_pitches[all_pitches.game_type == "R"][500000:550000].copy()
raw_data = z.copy()
cleaned_data = clean_raw_pitches(raw_data)

coef_dicts = build_neutralization_coefficient_dictionaries(cleaned_data)

neutralized_data = neutralize_stats(cleaned_data, coef_dicts)

rolled_stats = roll_neutralized_batting_stats(neutralized_data)

stitched_stats = stitch_pitbat_stats(rolled_stats)

final_dataset = merge_pitching_batting_leagueaverage_and_weather_datasets(stitched_stats, cleaned_data)

training_dataset = make_dataset_machine_trainable(final_dataset)

## To Do:
- save training output with google cloud saver
- run training script on 2016 full (and eventually 2016-19)
- re-run algorithms
- re-name and shape folders for github